In [1]:
!pip install datasets transformers rouge-score nltk

     |████████████████████████████████| 290 kB 4.2 MB/s 
     |████████████████████████████████| 2.9 MB 34.8 MB/s 
     |████████████████████████████████| 243 kB 51.2 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 125 kB 39.3 MB/s 
     |████████████████████████████████| 1.3 MB 41.2 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
     |████████████████████████████████| 3.3 MB 33.2 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 271 kB 45.8 MB/s 
     |████████████████████████████████| 160 kB 52.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import datasets
import numpy as np
import os
import pandas as pd
import re
import torch

In [4]:
os.chdir('/content/drive/MyDrive/CSE576')
print(os.getcwd())

/content/drive/MyDrive/CSE576


In [5]:
path = '/content/drive/MyDrive/CSE576/patient_train.csv'
df = pd.read_csv(path) 

In [6]:
df.head()

,symptoms,sentences
0,stomach ache,"Hello, there is a dull pain around the belly b..."
1,Bloating,"The appetite is also good, with a little bloat..."
2,diarrhea,It seems a bit like diarrhea. After I have eat...
3,"nausea, Dizziness",My subject was a 42-year-old male. He began to...
4,stomach ache,"My stomach hurts a bit before, but now I'm up."


In [7]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'patient_train.csv', 'test': 'patient_test.csv'})

Using custom data configuration default-695ef9fc34733887


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-695ef9fc34733887/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from datasets import load_metric
metric = load_metric("rouge")

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [9]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

In [15]:
model_checkpoint = "t5-small"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
tokenizer(df['sentences'][0])

{'input_ids': [8774, 6, 132, 19, 3, 9, 19511, 1406, 300, 8, 17719, 2218, 6, 27, 278, 31, 17, 214, 125, 31, 7, 352, 30, 41, 371, 15, 13513, 6, 2838, 203, 625, 61, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [12]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["symptoms"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    outputs = [output for output in examples["sentences"]]
    labels = tokenizer(outputs, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(dataset['train'][:1])

{'input_ids': [[21603, 10, 9883, 3, 4933, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1]], 'labels': [[8774, 6, 132, 19, 3, 9, 19511, 1406, 300, 8, 17719, 2218, 6, 27, 278, 31, 17, 214, 125, 31, 7, 352, 30, 41, 371, 15, 13513, 6, 2838, 203, 625, 61, 1]]}

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-dialogue-generation",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: symptoms, sentences.
***** Running training *****
  Num examples = 36454
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6837


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
Saving model checkpoint to t5-small-finetuned-dialogue-generation/checkpoint-500
Configuration saved in t5-small-finetuned-dialogue-generation/checkpoint-500/config.json
Model weights saved in t5-small-finetuned-dialogue-generation/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-dialogue-generation/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-dialogue-generation/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [t5-small-finetuned-dialogue-generation/checkpoint-1000] due to args.save_total_limit
Saving mo

In [ ]:
for index, row in df[:10].iterrows():
    inputs = row["symptoms"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
    summary_ids = model.generate(model_inputs['input_ids'].cuda())
    print(f'Symptoms:{row["symptoms"]}')
    print(f'Sentences:{row["sentences"]}')
    print(f'Generate:{tokenizer.decode(summary_ids[0], skip_special_tokens=True)}')
    print("\n")

Symptoms:stomach ache
Sentences:Hello, there is a dull pain around the belly button, I don't know what's going on (Female, 29 years old)
Generate:stomachache


Symptoms:Bloating
Sentences:The appetite is also good, with a little bloating.
Generate:Bloating


Symptoms:diarrhea
Sentences:It seems a bit like diarrhea. After I have eaten at noon1, I want to pull it after a while, a little pull.
Generate:diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea diarrhea


Symptoms:nausea, Dizziness
Sentences:My subject was a 42-year-old male. He began to feel dizzy this morning, and later became nauseous, bloated, and felt uncomfortable. Did he catch a cold while driving on the air conditioner? Or enteritis (female, 38 years old)
Generate:I feel nausea, dizziness, nausea, dizziness, nausea, dizziness


Symptoms:stomach ache
Sentences:My stomach hurts a bit before, but now I'm up.
Generate

In [11]:
from transformers import AutoTokenizer
path = '/content/drive/MyDrive/CSE576/t5-small-finetuned-dialogue-generation/checkpoint-10500'
tokenizer = AutoTokenizer.from_pretrained(path, local_files_only=True)

In [ ]:
prefix = "summarize: "

In [17]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [19]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(path, local_files_only=True)
model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [21]:
path = '/content/drive/MyDrive/CSE576/doctor_test.csv'
df = pd.read_csv(path)

In [22]:
for index, row in df[:10].iterrows():
    inputs = row["symptoms"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
    summary_ids = model.generate(model_inputs['input_ids'].cuda())
    print(f'Symptoms:{row["symptoms"]}')
    print(f'Sentences:{row["sentences"]}')
    print(f'Generate:{tokenizer.decode(summary_ids[0], skip_special_tokens=True)}')
    print("\n")

Symptoms:fever
Sentences:Do you have a fever.
Generate:Hello, do you have fever?


Symptoms:Bloating, fever, Vomit, Hiccup, Reflux
Sentences:Do you have other symptoms such as abdominal distension, vomiting, hiccups, pantothenic acid, and fever?
Generate:Do you have acid reflux, bloating, fever, vomiting, hiccups


Symptoms:Gastrointestinal disorders
Sentences:Your situation is considered to be caused by gastritis or gastrointestinal dysfunction. It is recommended to eat lightly, keep warm, do not eat spicy, greasy and raw or cold foods, drink more hot water, do not smoke or drink, exercise after meals, and you can eat Trimebutine and rabeprazole can improve the symptoms together. If it doesn't work, you need to go to the hospital for abdominal ultrasound and gastroscopy.
Generate:Symptoms of intestinal dysfunction are caused by intestinal dysfunction.


Symptoms:Gastrointestinal disorders
Sentences:Gastroenteritis or gastrointestinal dysfunction can be caused, and the specific needs t